# **Topic modelling with *kiara***
In this tutorial, you will learn how to use the state-of-the-art data orchestration sofware *kiara* to perform topic modelling on a large-scale corpus.

#**Digital research** 
As digital methods are more and more used in research, the need for transparency and critical reflection is ever more pressing. Regardless of the type of digital analysis that is performed or even the software that is used, documenting the *process* and not just the product becomes integral to the validity of the research findings. 

Despite a fast changing technological landscape, however, there is a frustrating lack of tools that allow you to document the process of digital knowledge production. In other words, how can we keep track of our methodological decisions and interventations?

# **Introducing *kiara***
*kiara* is a state-of-the-art sofware that allows researchers to document and critically reflect not solely on the digital methods and approaches they use, but perhaps more importantly also on how those methods and choices impact on the sources. When you begin using *kiara*, your research choices and changes to the data are recorded so that you will be able to visualise and examine the individual steps you took and how your data changed accordingly. This is important for the researcher, but also for reproducibility and replicability purposes.

*kiara* currently features several digital research approaches such as textual analysis and network analysis. In this tutorial, we will cover **Natural Language Processing** (NLP). 

For more information and updates on *kiara*, please check the [project's repository](https://github.com/DHARPA-Project/kiara).

# **Why NLP?**

Natural language processing technology allows researchers to sort through unstructured data such as plain text. In other words, by adding numerical value to text, computers can 'understand' language and perform advanced operations such as text categorisation, labelling, summarisation and so on. There are two main stages in NLP: pre-processing and analysis (aka, algorithm development and/or implementation). Here we will cover both stages; for the pre-processing part, we will use as an example some of the most common pre-processing operations such as tokenisation, lowercasing, removing stopwords, etc. For the analysis, we will use the example of another widely used text analysis method called **Topic Modelling**. For more information about the pre-processing operations and topic modelling and a more in-depth discussion particularly for humanities research, please refer to this repository [here](https://github.com/DHARPA-Project/TopicModelling-).

# **The case study: narratives of migration by Italian Americans, 1898-1936**
This tutorial uses *kiara* to examine how the changing experience of migration, identity construction, and assimilation is reflected over time in the accounts of migrants themselves. Using a corpus of Italian American newspapers, the analysis will allow us to address the fundamental question whether ethnic media facilitate assimilation and integration, or rather isolate immigrants from their new society by keeping them in the cultural sphere of their homeland (Parks, 2014).
The case study used for this tutorial is partially based on [Viola and Verheul (2019)](https://academic.oup.com/dsh/article/35/4/921/5601610#209907403).

# **Before starting**
For this tutorial, you will need a basic understanding of Python and how to run Python code, whether that be via Jupyter Notebooks or using a text editor and the command line. You will also need to install *kiara*. Please refer to the installation procedure [here](http://dharpa.org/kiara.documentation/latest/workshop/workshop/).

In [ ]:
%%capture
! pip install git+https://github.com/DHARPA-Project/kiara_plugin.dh_benelux_2023

# **Preparing the environment**
In order to use *kiara*, we first need to create a KiaraAPI instance. An API allows us to control and interact with *kiara* and its functions. In *kiara* this also allows us to obtain information on the available operations and on how these operations impact on the sources. For more information about the KiaraAPI, see the *kiara* API documentation [here](https://dharpa.org/kiara/latest/reference/kiara/interfaces/python_api/__init__/#kiara.interfaces.python_api.KiaraAPI).

In [ ]:
from kiara.api import KiaraAPI

kiara = KiaraAPI.instance()

We can now ask *kiara* to list all the operations that are included with the plugins we installed in the previous step.

In [ ]:
kiara.list_operation_ids()

# **Data onboarding**
In this tutorial, we will be using a sample from [*ChroniclItaly 3.0*](https://zenodo.org/record/4596345#.ZFJaGnZBw2w) (Viola and Fiscarelli 2021, Viola 2021), an open access digital heritage collection of Italian immigrant newspapers published in the United States from 1898 to 1936. This corpus includes the digitized (OCRed) front pages of ten Italian American newspapers as collected from [*Chronicling America*](https://chroniclingamerica.loc.gov/), an Internet-based, searchable database of U.S. newspapers published in the United States from 1789 to 1963 made available by the Library of Congress. In this tutorial, the sample we are using contains the mainstream newspaper *L'Italia* and the anarchic newspaper *Cronaca Sovversiva*. Using topic modelling to compare the topics discussed by Italian American migrants in the two newspapers of different political orientation will offer insights on how migration was experienced by the diverse Italian community, but also how it was narrated to the migrants themselves.     

The corpus is also a good example of how *kiara* can help us use metadata for historical research. The filenames of the files in *ChroniclItaly 3.0* already contain important metadata information such as the publication date and the publication reference number. The file name structure is:

LCCNnumber_date_pageNumber_ocr.txt

Therefore, the file name ‘sn84037025_1917-04-14_ed-1_seq-1_ocr.txt ’ refers to the OCR text file of the first page of the first edition of the title *La Rassegna* published on 14 April 1917. If your files also contain metadata information, *kiara* allows you to retrieve both the files and the metadata in the filenames. This is very useful for historical research, but also to keep track of how we are intervening on our sources. Let's see how this works.

In [ ]:
inputs = {
    "url": "https://github.com/DHARPA-Project/kiara.examples/archive/refs/heads/main.zip",
    "sub_path": "kiara.examples-main/examples/workshops/dh_benelux_2023/data"
 }

dl_bundle = kiara.run_job('download.file_bundle', inputs=inputs)
dl_bundle

Great, we've successfully imported a bundle of files. This has given us both the name of the folders in which we have our files (in this case, the newspaper's title), and the files themselves. As you can see, *kiara* also gives us additional information, for example how many files we have imported and the space the files occupy. This information may be useful to keep track of how we have changed our files after intervening on them. But we can also push this further and ask kiara to give us more information on the composition of the files. For now, let's give them some structure. *kiara* provides us with meta information on each of its functions. This built-in documentation is also helpful for teaching purposes. we can access this in-built documentation at any time by doing `kiara.list_operation_ids()`

In [ ]:
kiara.list_operation_ids()

To give structure to our files, we could create a dataframe to contain our files and metadata. Now that we have a list of all the possible operations available in *kiara*, let's have a look at `create.table.from.file_bundle` more closely. 

In [ ]:
kiara.retrieve_operation_info('create.table.from.file_bundle')

Now that we know what `create.table.from.file_bundle` does, we can use it on our sources.

In [ ]:
inputs = {
    'file_bundle' : dl_bundle['file_bundle']
}

corpus_table = kiara.run_job('create.table.from.file_bundle', inputs=inputs)
corpus_table

*kiara* has taken all the information from our files and added it to a table thus making it a bit easier to navigate. But *kiara* can also perform other meta-analytical operations on our files to enhance our understanding of our sources, for example it can extract metadata information. Let's see how this works. 

# **Extracting metadata**
When working with historical sources, the possibility to extract metadata information from the files can be a very valuable feature. Currently, kiara is able to extract metadata from strings that comply with the Library of Congress (LCCN) catalogue pattern: '/LCCN/yyyy-mm-dd/'. This adds the publication reference (i.e., LCCN) and the dates to a table as two new columns. Moreover, if a mapping scheme is provided between publication references and publication, kiara will add a column with the publication names. The mapping scheme should look like this:

[["LCCN","LCCN","LCCN"],["name","name","name"]]

Let's have a look at this specific module.

In [ ]:
kiara.retrieve_operation_info('get_lccn_metadata')

If your files also comply with the LCC, give it a try, if not, you can skip this step.

In [ ]:
inputs = {
    'table_input' : corpus_table['table'],
    'column_name' : 'rel_path',
    'map': [['2012271201','sn85054967','sn93053873','sn85066408','sn85055164','sn84037024','sn84037025','sn84020351','sn86092310','sn92051386'],['Cronaca_Sovversiva','Il_Patriota','L\'Indipendente','L\'Italia','La_Libera_Parola','La_Ragione','La_Rassegna','La_Sentinella','La_Sentinella_del_West','La_Tribuna_del_Connecticut']]
}

corpus_metadata = kiara.run_job('get_lccn_metadata', inputs=inputs)
corpus_metadata

Great! Now we know the publication name and the date of each individual file. This information can be very valuable particularly in topic modelling. For example, here we want to analyse if newspapers of different political orientation pushed different narratives of migration threfore, knowing the publication title can bring important insights in this respect. 

Similarly, knowing how many words each document has can be very important for linguistic analysis but also to keep track of how we change our files as we proceed with our analysis. Let's have a look at this specific module.

# **Exploring the files**
In the first step of NLP, texts are typically heavily pre-processed as we will see in the next steps of this tutorial. The idea is that the machine will be able to capture more information if we intervene with operations than if we didn't. However, every time we perform a pre-processing operation on our sources, we in fact may lose significant amounts of information. Keeping track of how much we have to 'sacrifice' for the sake of the computational analyisis can raise awareness of the challenges of digital analyisis.
As ususal, let's first call the meta-information available in *kiara*.  

In [ ]:
kiara.retrieve_operation_info('get_text_stats')

Let's now run this module on our corpus.

In [ ]:
inputs = {
    'table_input' : corpus_metadata['table_output'],
    'column_name' : 'content'
}

text_stats = kiara.run_job('get_text_stats', inputs=inputs)
text_stats

Great! Now we have a much clearer picture of how our sources look like. For example, we have information on how many words and characters each files has and the publication date and title of each file. This metadata is very important for traceability purposes but also for transparency and reproducibility. 

As *kiara* runs in Python, we can also export our *kiara* outputs into Python packages. For example, we can export our *kiara* table into a pandas dataframe to visualise our meta-findings. These steps will not be recorded by *kiara* as technically we are not impacting on the data. Nevertheless, these operations may bring valuable insights into our sources composition.

In [ ]:
table_obj = text_stats['table_output'].data
arrow_table = table_obj.arrow_table
sources = arrow_table.to_pandas()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set(font_scale=.7)

In [ ]:
f, axes = plt.subplots(1, 2)
sns.histplot(sources.chars_count, bins=10, ax=axes[0]).set(title='Characters distribution accross documents', xlabel='Number of characters', ylabel='Number of documents')
sns.histplot(sources.words_count, bins=10, ax=axes[1]).set(title='Words distribution accross documents', xlabel='Number of words')
plt.show()

Now we know that most of our documents contain between 3,000 and 4,000 words and around 20,000 characters. This is important from a pragmatic point of view: as topic modelling is essentially a statistical method, the length of each modelled item, i.e., the document, does matter. Indeed, it makes a
difference when we draw our conclusions on the relevance of the topics. 

Another way to further explore our sources is to visualise how our files are distributed in our collection. This may be particularly valuable in the case of timestamped files to visualise how they distribute over time, for example to identify potential gaps in the sources. Depending on individual research questions, this feature can also be helpful to create a subset of the corpus. Let's see how this works in *kiara*.

In [ ]:
kiara.retrieve_operation_info('viz_data_query')

Let's try this module on our sources.

In [ ]:
inputs = {
    'distribution' : 'month',
    'column' : 'pub_name',
    'table': text_stats['table_output']
}

viz_data = kiara.run_job('viz_data_query', inputs=inputs)

To visualise the results, we need an aditional, external package called *Observable*. Let's import it and embed it into our workflow.

In [ ]:
from observable_jupyter import embed

In [ ]:
embed('@dharpa-project/timestamped-corpus', cells=['viewof chart', 'style'], inputs={"data":viz_data['viz_data'].data.list_data,"scaleType":'height', "timeSelected":'month'})

Wonderful! Now we know immediately that although our corpus on the whole spans from December 1896 to December 1920, the two newspapers' titles are not perfectly comparable. Specifically, *Cronaca Sovversiva* covers from May 1903 to May 1919 (with two gaps in 1918 and 1919). *L'Italia* also presents gaps in 1898 and 1899. We can also immediately see how many files we have in each month for each title, which will allow us to moderate our conclusions accordingly.

Now that we know how our files are distributed in our corpus, we may consider creating a subset so as to obtain a more comparable sample. Let's do that.

In [ ]:
kiara.retrieve_operation_info('query.table')

Let's first create our specific time span in the form of a query. As we want to create a comparable sample, let's select a timeframe which includes files from both titles (i.e., from 06.06.1903 and finish 06.06.1905) 

In [ ]:
date_ref_1 = "1903-6-6"
date_ref_2 = "1905-6-6"
query = f"SELECT * FROM sources WHERE date <= DATE '{date_ref_2}' AND date > DATE '{date_ref_1}'"

In [ ]:
inputs = {
    'query' : query,
    'table': text_stats['table_output']
}

subset = kiara.run_job('query.table', inputs=inputs)
subset

Excellent! Now we are all set to start pre-processing our sources, that is Stage 1 of of NLP.

# **Natural Language Processing (Stage 1)**
Now we are ready for preparing our texts for analysis, that would be stage 1. Let's see what operations are included in *kiara* for NLP in the `kiara_plugin.language_processing package`.

In [ ]:
infos = kiara.retrieve_operations_info()
operations = {}
for op_id, info in infos.item_infos.items():
    if info.context.labels.get("package", None) == "kiara_plugin.language_processing":
        operations[op_id] = info

print(operations.keys())

In order to process and analyse our sources, we need to work with the textual content in the column 'content' of the table we just created. Let's explore the available operations specific for tables by running `kiara.list_operation_ids('table')`.

In [ ]:
kiara.list_operation_ids('table')

As we are interested in one column, the `table.pick.column` operation seems like a good fit.

In [ ]:
inputs = {
    'table': subset['query_result'],
    'column_name': 'content'
}

text_contents = kiara.run_job('table.pick.column', inputs=inputs)
text_contents

Before performing any operation, we should start by tokenising our text. Tokenisation is a process that tells the machine at which level we want to perform our operations (e.g., character level, word level, sentence level). In other words, by tokenising we define the boundaries of the elements in our texts. 

If you're unsure about which ones of these operations you should run, you can refer to the in-built explanation in each *kiara* module or refer to this repository [here](https://github.com/DHARPA-Project/TopicModelling-) for further information about pros and cons of each pre-processing operation.

In [ ]:
kiara.retrieve_operation_info('tokenize.texts_array')

As we will later perform topic modelling, let's tokenise our texts at word level.

In [ ]:
inputs = {
    'texts_array': text_contents['array']
}

tokens = kiara.run_job('tokenize.texts_array', inputs=inputs)
tokens

Now that the text of our files is tokenised, we can experiment with different pre-processing operations. Again, we can use the `kiara.retrieve_operation_info` to explore what *kiara* can do.

In [ ]:
kiara.retrieve_operation_info('preprocess.tokens_array')

*kiara* includes the most widely used text analysis pre-processing operations. Let's try some of them and take a few moments to notice how they change our texts.

Let's start by removing stopwords. These are low information words, i.e., not semantically salient such as articles, pronouns, prepositions, conjunctions, etc. There are numerous stopword lists available for many, though not all, languages which can be easily adapted to the individual researcher's needs. Here we are defining our stopword list but feel free to experiment with your own.

Let's retrieve our stopword list from our kiara's GitHub repository. If you have your own stopword list, you can define it as a variable and skip this step.

In [ ]:
! wget https://raw.githubusercontent.com/DHARPA-Project/kiara.examples/main/examples/workshops/dh_benelux_2023/notebook/stop_words.csv

In [ ]:
import pandas as pd
custom_stopword_list = list(pd.read_csv('stop_words.csv')['stopword'])

Now that we have defined our stopword list, let's tell *kiara* to use it. 

In [ ]:
inputs = {
    "languages": ["italian", "english"],
    "stopwords": custom_stopword_list
}

stopwords_outputs = kiara.run_job('create.stopwords_list', inputs=inputs)
my_stopwords_list = stopwords_outputs['stopwords_list']
my_stopwords_list.data

As you can see, our customed stopword list contains Italian and English. This is tailored to the specificity of our sources, newspapers written by Italian Americans that are mostly in Italian but which also contain some English text. Using a customed list is preferable in most cases as it likely improves the quality of the results. 
We can now go ahead and ask *kiara* to remove the stopwords from our texts.

Other pre-processing operations that we can perform to potentially improve the quality of our results are lowercasing and remove all the tokens shorter or equal to a value that we set. Again, this is another example of interventions on the sources that depend on the researcher's choices and that are based on the specificity of the sources. In this case, as we are working with historical newspapers, the quality of the OCR content is not optimal, meaning that our texts contain several errors. Removing tokens shorter than or equal to X characters for example may remove many of such errors. Here we are definining this value at 2 but feel free to experiment with different options based on your sources.Again, it is important to keep in mind that each intervention on our sources heavily transforms them and as such, it has pros and cons. For this reason, although it is possible to combine more operations into one function, it is a good idea to proceed step-by-step and see how our choices are impacting on the material. For the purposes of this tutorial, let's combine lowercasing with removing any words with non-alpha tokens that is removing all tokens that include punctuation and numbers (e.g., ex1a.mple) and tokens that contain less than two characters.

In [ ]:
inputs = {
    'tokens_array': tokens['tokens_array'],
    'remove_stopwords' : my_stopwords_list,
    'to_lowercase': True,
    'remove_non_alpha': True,
    'remove_short_tokens': 2
}

preprocessed_tokens = kiara.run_job('preprocess.tokens_array', inputs=inputs)
preprocessed_tokens

Great! Another operation that we can perform to improve the quality of our topic model is to transform our corpus into a dictionary of n-grams, typically bi-grams or tri-grams. These are pairs or triples of words that are statistically more likely to occur together than if theywere found independently from each other. Several studies (see for instance Wallach, 2006; Wang et al., 2007; Kherwa and Bansal, 2020) have indeed reported that using bi-grams to prepare the corpus may increase topics’ interpretability as well as the efficiency of statistical methods such as perplexity and coherence developed to help researchers and practitioners optimise topic modelling results. Beware however that preparing the corpus as a dictionary of n-grams is a lengthy and intense process which may indeed be costly and time consuming, especially in the case of very large repositories.

In [ ]:
kiara.retrieve_operation_info('get_bigrams')

The built-in documentation gives us some insights on the parameters that the user can tweak and experiment with. Here we are defining these values as shown below but feel free to experiment with different combination. You as an expert of your sources are the best judge of which combination is more informative.

In [ ]:
inputs = {
    'tokens' : preprocessed_tokens['tokens_array'],
    'bigrams_threshold' : 20,
    'bigrams_min_count' : 3,
}

bigrams = kiara.run_job('get_bigrams', inputs=inputs)
bigrams

Excellent! Now we are all set to create our first topic model with *kiara*, that is Stage 2 of of NLP.

# **Natural Language Processing (Stage 2)**
Now we are ready for analysing our texts, that would be stage 2. Let's see what operations are included in *kiara* for topic modelling with LDA, that is the statistical package [Latent Dirichlet Allocation](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), in *kiara*.

In [ ]:
kiara.retrieve_operation_info('generate.LDA.for.tokens_array')

Research has been conducted towards implementing statistical methods that could
help researchers and practitioners find the craved ‘optimal number of topics’. Two of the most common methods are model perplexity and topic coherence, measures that score the statistical quality of different topic models based on the topics’ compositions in several models. Though not unanimously, the believed assumption behind these techniques is that a higher statistical quality yields more interpretable topics.

*kiara* currently implements Topic Coherence. What the method is designed to do is to model human judgement by scoring the composition of the topics based on how coherent, i.e., interpretable they are (Röder et al., 2015). If the coherence score increases as the number of topics increases, for example, that would suggest that the most interpretable model is the one that displays the highest coherence value before flattening out or dropping. Both techniques are widely used to determine the number of topics; the truth is, however, that neither of these measures is ideal because what they actually score is the probability of observations and not their degree of semantic meaning (Chang et al., 2009). This is why we can set this parameter as `False` if we prefer to use human judgement. For the purposes of the tutorial, let's keep the parameter as `True`. The algorithm will then compute six topic models, that is one model for each of the number of topics we set as our preferred interval, in this case all the models for topics between 5 and 10.

In [ ]:
inputs = {
    'tokens_array' : bigrams['tokens_array'],
    'num_topics_min' : 5,
    'num_topics_max' : 10,
    'compute_coherence': True,
}

lda = kiara.run_job('generate.LDA.for.tokens_array', inputs=inputs)
lda

Fantastic, we have created our first topic models! But which one is the most statistically accurate? As you can see in our previous output, *kiara* prints both the topics and a coefficient next to each model. The latter is the coherence score calculated for the respective number of topics. That's all wonderful but not really human readable. A helpful way to navigate the results may be to plot the coherence score, let's give it a try.

In [ ]:
coherence_map = lda['coherence_map'].data.dict_data

In [ ]:
sns.lineplot(coherence_map)

The coherence map helps us better understand which model is most statistically reliable. If the coherence score increases as the number of topics increases, for example, that would suggest that the most interpretable model is the one that gave the highest coherence value before flattening out. So theoretically, the most accurate model in this case that would be topic model n. 8. 

In [ ]:
topics_words = lda['topic_models'].data.dict_data['8']
topics_words

We have almost reached the end of our tutorial. We have accomplished a lot, we have explored our sources, intervened with several customed operations, created new data, visualised them and experimented with topic modelling. Hopefully, we have also obtained insightful results from our analysis and learned a lot in the process. But how did we get here? Can we recall all the decisions we made, the turns and paths we took without going through all the cells from the top? *kiara* has tracked all that for us; we can access this log through the function `lineage`. Let's see how it works.

In [ ]:
topics = lda['topic_models']

topics.lineage

As we can see, *kiara* has given a unique id to each of the operations we ran in the order we performed them starting from the last one. But we can also visualise this log and see how that looks like in the form of a decision tree.

In [ ]:
from kiara_plugin.dh_benelux_2023.utils import augment_lineage_data

In [ ]:
augmented_nodes = augment_lineage_data(topics,kiara)

In [ ]:
from observable_jupyter import embed

In [ ]:
embed('@dharpa-project/kiara-data-lineage', cells=['displayViz', 'style'], inputs={'dataset':augmented_nodes, 'modulesNodes':12, 'valueNodes':7, 'Width': 300, 'Height': 500})